## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
           'OverTime', 'StockOptionLevel', 'WorkLifeBalance',
           'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes



Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Convert your X data to numeric data types however you see fit
X_df['OverTime'] = X_df['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)



<ipython-input-5-ce9b498d06dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df['OverTime'] = X_df['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)


In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)




In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)



In [8]:
# Create a OneHotEncoder for the Department column
encoder_dept = OneHotEncoder()

# Fit the encoder to the training data
encoder_dept.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept = encoder_dept.transform(y_train[['Department']]).toarray()
y_test_dept = encoder_dept.transform(y_test[['Department']]).toarray()



In [9]:
# Create a OneHotEncoder for the Attrition column
encoder_attr = OneHotEncoder()

# Fit the encoder to the training data
encoder_attr.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr = encoder_attr.transform(y_train[['Attrition']]).toarray()
y_test_attr = encoder_attr.transform(y_test[['Attrition']]).toarray()


## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
X_train_scaled.shape[1]


# Create the input layer
input_layer = Input(shape=(X_train_scaled.shape[1],), name='input')


# Create at least two shared layers
shared1 = Dense(64, activation='relu', name='shared1')(input_layer)
shared2 = Dense(128, activation='relu', name='shared2')(shared1)


In [11]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
dept_output = Dense(y_train_dept.shape[1], activation='softmax', name='department_output')(dept_hidden)



In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden = Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attr_output = Dense(y_train_attr.shape[1], activation='sigmoid', name='attrition_output')(attr_hidden)



In [13]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
model.compile(optimizer=Adam(),
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [14]:
# Train the model
history = model.fit(X_train_scaled, {'department_output': y_train_dept, 'attrition_output': y_train_attr},
                    validation_data=(X_test_scaled, {'department_output': y_test_dept, 'attrition_output': y_test_attr}),
                    epochs=100, batch_size=32)



Epoch 1/100
37/37 [==============================] - 3s 16ms/step - loss: 1.3707 - department_output_loss: 0.8788 - attrition_output_loss: 0.4919 - department_output_accuracy: 0.5969 - attrition_output_accuracy: 0.8274 - val_loss: 1.1684 - val_department_output_loss: 0.7868 - val_attrition_output_loss: 0.3816 - val_department_output_accuracy: 0.6667 - val_attrition_output_accuracy: 0.8673
Epoch 2/100
37/37 [==============================] - 0s 6ms/step - loss: 1.1906 - department_output_loss: 0.7840 - attrition_output_loss: 0.4066 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8316 - val_loss: 1.1687 - val_department_output_loss: 0.7862 - val_attrition_output_loss: 0.3825 - val_department_output_accuracy: 0.6667 - val_attrition_output_accuracy: 0.8673
Epoch 3/100
37/37 [==============================] - 0s 6ms/step - loss: 1.1556 - department_output_loss: 0.7697 - attrition_output_loss: 0.3859 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8367

In [16]:
# Evaluate the model with the testing data
evaluation = model.evaluate(X_test_scaled, {'department_output': y_test_dept, 'attrition_output': y_test_attr})


10/10 [==============================] - 0s 4ms/step - loss: 4.6225 - department_output_loss: 2.8864 - attrition_output_loss: 1.7361 - department_output_accuracy: 0.5170 - attrition_output_accuracy: 0.7891


In [17]:
# Print the accuracy for both department and attrition
print(f"Total Loss: {evaluation[0]},\nDepartment Loss: {evaluation[1]}, Attrition Loss: {evaluation[2]},\nDepartment Accuracy: {evaluation[3]}, Attrition Accuracy: {evaluation[4]}")


Total Loss: 4.622489929199219,
Department Loss: 2.8863539695739746, Attrition Loss: 1.7361361980438232,
Department Accuracy: 0.5170068144798279, Attrition Accuracy: 0.7891156673431396


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

### Addressing the Summary Discussion Questions:

1. **Is accuracy the best metric to use on this data? Why or why not?**

    - Accuracy might not be the best metric if the data is imbalanced, which is often the case with attrition prediction where the number of employees leaving might be significantly lower than those staying. Metrics like F1 score, precision, recall, or the area under the ROC curve (AUC) provide more insight into model performance, especially for imbalanced datasets. They help understand how well the model identifies true positives while managing false positives and false negatives.

2. **What activation functions did you choose for your output layers, and why?**

    - For the department classification (a multi-class problem), a `softmax` activation function was used in the output layer because it outputs a probability distribution over the classes, which suits multi-class classification tasks. For attrition (a binary classification problem), a `sigmoid` function was used because it outputs probabilities in a 0-1 range, ideal for binary outcomes.

3. **Can you name a few ways that this model might be improved?**
  - Using more sophisticated neural network architectures like recurrent neural networks (RNNs) if the data has a temporal component or if sequence prediction could be beneficial.
  - Exploring different loss functions that might be more appropriate for the specific characteristics of the dataset.
  - Implementing cross-validation to ensure that the model's performance is consistent across different subsets of the data, enhancing the model's generalizability.